# Robust LP with Polyhedron Cost Complexity

## Optimization Problem 



$\begin{gather}
\quad \text{minimize} \quad & \sup_{c \in \mathcal{C}} c^Tx \\
\quad \text{subject to} \quad & Ax \geq b \\
\end{gather}$
<br> <br>
$\text{where } \mathcal{C} = \{ c \ | \ Fc \leq g \}$

## (a) Convexity of $f(x) = \sup_{c \in \mathcal{C}} c^Tx$

$f(x) := c_x^Tx$ $\text{ such that }$ $c_x^Tx \geq c^Tx \ \forall \ c \in \mathcal{C}$

<br>

$\text{For some }$ $\theta \in [0, 1], $

$\text{It can be inferred that }$ $f(\theta x) = \theta c_x^T x$ $\text{ since }$ 
$\ \theta c_x^Tx \geq \theta c^Tx \ \forall \ c \in \mathcal{C}$

<br>

$\text{Suppose, }$ 
$f(\theta x + (1 - \theta)y) = c_\theta^T (\theta x + (1 - \theta)y)$

<br>

$\text{We need to prove Jensen's Inequality for Convexity }$

$f(\theta x + (1 - \theta)y) \ \leq \ \theta f(x) + (1-\theta)f(y) \ \forall \ \ \theta \in [0, 1]$

$\text{That is, }$ $c_\theta^T (\theta x + (1 - \theta)y) \ \leq \ \theta c_x^Tx + (1-\theta)c_y^Ty \ \forall \ \ \theta \in [0, 1]$

<br>

$\text{Since }$ $c_x^Tx \geq c_\theta^Tx \text{ and } c_y^Ty \geq c_\theta^Ty$,
$\text{ the above condition is satisfied and hence}$

$\mathbf{f(x) = \sup_{c \in \mathcal{C}} c^Tx}$ $\text{is Convex}$

<br><br>

## (b) Getting $f(x)$

$
f(x) = 
\left( \begin{gather*} 
\text{maximize} & c^Tx \\
\text{subject to} & Fc \leq g \\
\end{gather*} \right)
$

<br><br>

$
f(x) = 
\left( \begin{gather*} 
\text{minimize} & -x^Tc \\
\text{subject to} & Fc - g \leq 0 \\
\end{gather*} \right)
$

<br><br>

$
\text{Lagrangian}, \ L(x, \lambda_+) = -x^T c + \lambda_+^T(Fc-g) = -g^T \lambda_+ + (F^T \lambda_+ - x)^T c \quad \quad \leq -x^Tc \\
\text{Dual}, \ g(\lambda_+) = \underset{x}{\inf} L(x, \lambda_+) = -g^T \lambda_+ + \underset{x}{\inf} \ (F^T \lambda_+ - x)^T c \\
g(\lambda_+) \ = \ -g^T \lambda_+ \quad \text{ if } \quad (F^T \lambda_+ - x)^T = 0 \quad \text{ and } \quad \lambda_+ \geq 0 \\
g(\lambda_+) \ = \ -\infty \quad \text{ otherwise } \quad  \text{(Ignored as we need to maximize } \ L(x, \lambda_+) \text{)} \\
$


<br><br>

$
\text{Therefore, } \\
f'(x) = 
\left( \begin{gather*} 
\text{maximize} & -g^T \lambda_+ \\
\text{subject to} & F^T\lambda_+ - x = 0 \\
\ & \lambda_+ \geq 0
\end{gather*} \right)
$

$
\text{is the dual of the function } f(x) \text{ and the optimal value is } f(x)
$

<br><br>

$
\text{Therefore, } \\
f(x) = 
\left( \begin{gather*} 
\text{minimize} & g^T \lambda_+ \\
\text{subject to} & F^T\lambda_+ - x = 0 \\
\ & \lambda_+ \geq 0
\end{gather*} \right)
$

<br><br>

## (c) Formulating using $f(x)$


$
\text{Our optimization problem is, } \\
\begin{gather}
\quad \text{minimize} \quad & f(x) \\
\quad \text{subject to} \quad & Ax \geq b \\
\end{gather}$


$\text{where}$

$
f(x) = 
\left( \begin{gather*} 
\text{minimize} & g^T \lambda_+ \\
\text{subject to} & F^T\lambda_+ - x = 0 \\
\ & \lambda_+ \geq 0
\end{gather*} \right)
$

<br><br>

$ 
\text{Our optimization problem can be written as } \\
\left(
\begin{align}
\quad \text{minimize} \quad & \quad \quad g^T \lambda_+ \\
\quad \text{subject to} \quad & F^T\lambda_+ = \ x, \quad \lambda_+ \geq 0, \quad Ax \ \geq \ b \\
\end{align}
\right)
$

<br><br>

## (d) Solving the Robust LP

### Given

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

np.random.seed(10)
(m, n) = (30, 10)
A = np.asmatrix(np.random.rand(m, n))
b = np.asmatrix(np.random.rand(m, 1))
c_nom = np.asmatrix(np.ones((n, 1)) + np.random.rand(n, 1))

### Converting the given constraints to $Fc \ \leq \ g$

In [2]:
F = np.r_[
          np.identity(n),  # For c <= 1.25c_nom
          -np.identity(n),  # For -c <= -0.75c_nom
          np.ones((1, n)) / n,  # For 1' @ c/n <= 1.1 (1' @ c_nom) / n
          -np.ones((1, n)) / n  # For - 1' @ c/n <= -0.9 (1' @ c_nom) / n
]

g = np.r_[
          1.25 * c_nom,
          -0.75 * c_nom,
          1.1 * np.ones(n) @ c_nom / n,
          -0.9 * np.ones(n) @ c_nom / n
]

### Implementation

In [3]:
x = cp.Variable((n, 1))
lam_pos = cp.Variable((len(g), 1))

objective = cp.Minimize(lam_pos.T @ g)
constraints = [
               A@x >= b,
               F.T@lam_pos == x,
               lam_pos >= 0
]

# Calculating x_nom Minimizing c_nom.T @ x with Ax >= b
prob = cp.Problem(cp.Minimize(c_nom.T @ x), constraints[0:1])
result_nom = prob.solve()
x_nom = np.matrix(x.value)

# Using Robust Constraints
# f(x_rob) gives the worst case f(x)
prob = cp.Problem(objective, constraints)
result_rob = prob.solve()  
x_rob = np.matrix(x.value)

# For Nominal Case, we don't have the worst case f(x)
# We use x_nom and calculate Worse case f(x) using the constraint on c
c = cp.Variable((n, 1))
objective = cp.Maximize(c.T @ x_nom)
constraints = [
               c <= 1.25 * c_nom,
               0.75 * c_nom <= c,

               c.T @ np.ones(n) / n <= 1.1 * np.mean(c_nom),
               0.9 * np.mean(c_nom) <= c.T @ np.ones(n)
]
fx_nom_worst = cp.Problem(objective, constraints).solve()

print("C-nom @ X: ")
print("Without using Robust Constraints:", np.sum(c_nom.T@x_nom))
print("With using Robust Constraints:", np.sum(c_nom.T@x_rob))

print()

print("Worst Case f(x): ")
print("Without using Robust Constraints:", fx_nom_worst)
print("Using the Robust Constraints:", result_rob)

C-nom @ X: 
Without using Robust Constraints: 2.1092714620826003
With using Robust Constraints: 2.5232088649062265

Worst Case f(x): 
Without using Robust Constraints: 7.221562204118721
Using the Robust Constraints: 3.16596105233182
